In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim  
import torchvision.transforms as transforms
import torchvision
import os
from torchvision.io import decode_jpeg
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets,models
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.io import read_image

In [ ]:
PATH = './'
NUM_CLASSES = 10
BATCH_SIZE = 32

In [ ]:
class CreateDataset(Dataset):
    def __init__(self, df,transform=False):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):   
        img_path = self.df.iloc[index, 0]
        image = read_image(img_path) / 255.0
        label = self.df.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
transformers = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

train_dataset=CreateDataset(train_df, transformers)
test_dataset=CreateDataset(test_df, transformers)
val_dataset=CreateDataset(val_df, transformers)

In [ ]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
import pytorch_lightning as pl
import torchmetrics
from torch import nn


class Model(pl.LightningModule):
    def __init__(self, output_units, learning_rate):
        super().__init__()
        self.base_model = torchvision.models.resnet50(pretrained=True)
        
        freezing_layers = [
            self.base_model.conv1,
            self.base_model.bn1,
#             self.base_model.layer1,
#             self.base_model.layer2,
#             self.base_model.layer3,
        ]
        
        for layer in freezing_layers:
            for param in layer.parameters():
                param.requires_grad = False
    
                
        self.base_model.fc = torch.nn.Linear(in_features=self.base_model.fc.in_features, out_features=output_units)
        
        self.criterion = nn.CrossEntropyLoss()
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()

        self.learning_rate = learning_rate
        self.save_hyperparameters()
        
    def forward(self, input_data):
        return self.base_model(input_data)

    def training_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        loss = self.criterion(preds, targets)
        self.log('train_loss', loss)
        self.train_acc(preds, targets)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=False, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        loss = self.criterion(preds, targets)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.val_acc(preds, targets)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True)
        
    def test_step(self, batch, batch_nb):
        self.validation_step(batch, batch_nb)
        
    def predict_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        return torch.argmax(preds, dim=1)
    
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=1e-2)

In [ ]:
model = Model(NUM_CLASSES, 5e-4)
callbacks = [
    pl.callbacks.ModelCheckpoint(monitor='val_acc', dirpath=PATH + 'Models/', verbose=True, mode='max', filename='resnet50-t3-{val_acc:.4f}'),
    pl.callbacks.EarlyStopping(monitor='val_acc', patience=15, verbose=True, mode='max')
]

trainer = pl.Trainer(max_epochs=100, callbacks=callbacks, gpus=1)    
trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
trainer.test(dataloaders=test_dataloader)

In [ ]:
trainer.test(dataloaders=train_dataloader)

In [ ]:
y_pred = trainer.predict(dataloaders=val_dataloader)

In [ ]:
y_pred_all = []
for y in y_pred:
    y_pred_all.extend(y.cpu().tolist())

In [ ]:
val_df['pred'] = y_pred_all

val_df.to_csv('val_pred.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(val_df['label'], val_df['pred'])